In [6]:
import pandas as pd, numpy as np, mglearn, numpy.linalg as nl
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tqdm import tqdm
from matplotlib.pyplot import *
from sklearn.decomposition import TruncatedSVD

In [4]:
#Preparing data
train_full = pd.read_csv('Data/spam_train.csv',index_col=0)

In [5]:
#General Functions
def make_knn(neighbor):
    return KNeighborsClassifier(n_neighbors=neighbor)
def make_log(C):
    return LogisticRegression(C=C)
def make_svm(C):
    return SVC(C=C)

def basic_analysis(model, train_data,test_data, attr_name='attribute', save=False, out_dir = ''):
    #Normalize data beforehand
    model.fit(train_data[0],train_data[1])
    

def cval_helper(model_gen, train_data, test_data, save=False, outdir=''):
    #Normalize data beforehand, place them into train_data


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_%3B,char_freq_%28,char_freq_%5B,char_freq_%21,char_freq_%24,char_freq_%23,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
3009,0.07,0.00,0.07,0.0,0.00,0.00,0.00,0.46,0.69,0.00,...,0.142,0.175,0.032,0.000,0.021,0.000,3.007,60,791,0
798,0.00,0.00,0.74,0.0,1.85,0.37,0.37,0.00,0.00,0.74,...,0.000,0.264,0.000,0.000,0.000,0.000,2.492,12,172,1
4396,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.763,0.000,2.181,6,24,0
353,0.00,0.00,0.32,0.0,0.65,0.65,0.32,0.32,0.00,0.00,...,0.000,0.105,0.000,0.000,0.158,0.000,1.494,10,139,1
574,1.03,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.176,0.176,0.000,2.766,26,83,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1,4,0
602,0.00,0.58,0.58,0.0,0.00,0.00,0.00,0.58,0.58,7.55,...,0.000,0.192,0.000,0.192,0.000,0.096,1.526,10,58,1
167,0.00,0.71,0.35,0.0,0.35,0.00,0.00,0.00,0.00,0.00,...,0.000,0.058,0.000,0.700,0.000,0.000,3.768,61,260,1
1314,0.54,0.13,0.38,0.0,0.05,0.19,0.00,0.05,0.35,0.16,...,0.000,0.086,0.000,0.273,0.150,0.159,6.789,195,3327,1
